In [ ]:
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16

base = VGG16(
    weights="./Object Detection(Ass6)/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",
    include_top=False,
    input_shape=(224,224,3)
)

# ---- Freeze base model ----
for layer in base.layers:
    layer.trainable = False

# Create ImageDataGenerator for training and validation
train_gen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2
)

selected_classes = ['airplanes', 'ant']

train_data = train_gen.flow_from_directory(
    "./Object Detection(Ass6)/caltech-101-img",
    target_size=(224,224),
    batch_size=32,
    subset='training',
    classes=selected_classes
)

val_data = train_gen.flow_from_directory(
    "./Object Detection(Ass6)/caltech-101-img",
    target_size=(224,224),
    batch_size=32,
    subset='validation',
    classes=selected_classes
)

# ---- Add Classifier on top of base model ----
model = Sequential([
    base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(train_data.num_classes, activation='softmax')
])

# ---- Compile (train only new layers) ----
model.compile(
    optimizer=Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# ---- Train ----
model.fit(train_data, validation_data=val_data, epochs=5)


loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")